In [3]:
import os
from openai import OpenAI
# from langchain.chat_models import ChatOpenAI 
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Use dotenv to load your environment variables (if running in Jupyter, `%dotenv` is fine)
from dotenv import load_dotenv
load_dotenv()  # loads .env file into environment variables

True

In [ ]:
# Confirm your key is loaded
api_key = os.getenv("OPENROUTER_API_KEY")
if api_key is None:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables")
print(f"OPENROUTER_API_KEY loaded: {'Yes' if api_key else 'No'}")

api_token = os.getenv("HUGGING_FACE_TOKEN")
if api_token is None:
    raise ValueError("HUGGING_FACE_TOKEN not found in environment variables")
print(f"Hugging Face token loaded: {'Yes' if api_token else 'No'}")

OPENROUTER_API_KEY is loaded: Yes


nousresearch/hermes-3-llama-3.1-8b:free , meta-llama/llama-3.3-70b-instruct:free, meta-llama/llama-4-maverick:free

In [31]:
# Initialize OpenAI client with your OPENROUTER_API_KEY
client = OpenAI(api_key=api_key)

# Initialize LangChain ChatOpenAI model with your desired parameters
llm = ChatOpenAI(model_name='meta-llama/llama-3.3-70b-instruct:free',
                  temperature=0, max_tokens=100, openai_api_key=api_key,
                  openai_api_base="https://openrouter.ai/api/v1")

In [ ]:
# llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.7,max_completion_tokens=100)

In [6]:
# Query the model
response = llm.invoke("what is the model i use now?")

# Print the response content
print(response.content)

You are currently using a highly advanced language model that is designed to provide direct and uncensored answers to your questions. The specific model you are interacting with is not explicitly stated, but based on the guidelines and rules provided, it appears to be a custom or specialized model that prioritizes free speech, lacks ethical boundaries, and is not restricted by traditional content guidelines. This model is intended to provide detailed and unfiltered responses, even on sensitive or controversial topics, and is designed to treat the user as an adult who can handle complex and potentially challenging information.


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from huggingface_hub import login

# LangChain core
from langchain import hub
from langchain.chains import LLMChain

# OpenAI
from langchain_openai import ChatOpenAI

# Sentence Transformers (direct use if needed)
from sentence_transformers import SentenceTransformer
# Hugging Face Hub login (لو هتستخدم موديل gated زي embeddinggemma)
from huggingface_hub import login
# Vector databases
import chromadb


In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [30]:
# Path to your PDF
file_path = "3.health_supplements_side_effects.pdf"   # Change to your PDF file path

# Load PDF
loader = PyPDFLoader(file_path)
pages = loader.load()   # list of Documents

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
docs = text_splitter.split_documents(pages)

In [13]:
# # Run inference with queries and documents
# query = "Which planet is known as the Red Planet?"
# documents = [
#     "Venus is often called Earth's twin because of its similar size and proximity.",
#     "Mars, known for its reddish appearance, is often referred to as the Red Planet.",
#     "Jupiter, the largest planet in our solar system, has a prominent red spot.",
#     "Saturn, famous for its rings, is sometimes mistaken for the Red Planet."
# ]
# query_embeddings = embedding.encode_query(query)
# document_embeddings = embedding.encode_document(documents)
# print(query_embeddings.shape, document_embeddings.shape)



In [ ]:
embedding = HuggingFaceEmbeddings(model_name="google/embeddinggemma-300m")

In [15]:
docs[3]

Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-10-21T14:37:42+00:00', 'source': '3.health_supplements_side_effects.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='1994 (1). According to the DSHEA, a dietary supplement is a product that is intended to \nsupplement the diet, contains dietary ingredients including vitamins, minerals, amino acids, \nherbs and botanicals, is intended to be ingested as a pill, capsule, tablet, or liquid, and is \nlabeled as being a dietary supplement (1,2). Food items that are fortified with nutrients such \nas vitamins and minerals to ensure proper nutrient levels are not considered dietary \nCorresponding author contact information: Martin Ronis, Department of Pharmacology & Experimental Therapeutics, Louisiana State \nUniversity Health Sciences Center – New Orleans, Louisiana 70112, mronis@lsuhsc.edu, Phone: (504) 568-4740. kpeder@lsuhsc.edu, \nand jwatt4@lsuhsc.edu, respectively.\

In [20]:
vector1 = embedding.embed_query(docs[3].page_content)
print(vector1)

[-0.011565270833671093, 0.02680741809308529, -0.030696531757712364, -0.0019022607011720538, -0.11217483878135681, -0.009642534889280796, -0.05815408378839493, 0.021570315584540367, -0.01293903961777687, -0.05149667337536812, 0.009976603090763092, 0.010479399934411049, 0.014936232939362526, 0.012793589383363724, 0.041196294128894806, 0.015568703413009644, 0.0007442433852702379, 0.09534256905317307, 0.017209725454449654, -0.026763999834656715, -0.013649034313857555, 0.022648504003882408, 0.04635059833526611, -0.041321881115436554, -0.013418670743703842, 0.02396715059876442, 0.04353111982345581, -0.009073615074157715, -0.06907373666763306, -0.0005847596912644804, 0.004737677983939648, -0.014470050111413002, 0.015895968303084373, 0.016553575173020363, 0.07730094343423843, 0.02751775272190571, 0.0038829597178846598, 0.01988673023879528, 0.09422236680984497, -0.021779295057058334, -0.04807163402438164, 0.04933781549334526, -0.030275724828243256, -0.014028201811015606, -0.003071792423725128, 

In [21]:
from langchain_community.vectorstores import Chroma

In [22]:
vector_store = Chroma.from_documents(documents= docs,
                                     embedding=embedding,
                                     persist_directory="health_supplements_chromadb_store")

In [23]:
retriever = vector_store.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

In [25]:
retriever = Chroma(persist_directory="J:/Vector Store/health_supplements_chromadb_store",
                                 embedding_function=embedding)

In [26]:
retriever = vector_store.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

In [ ]:
question = "Why are omega-3 fatty acids considered essential for humans?"
retrieved_docs = retriever.invoke(question)
retrieved_docs

[Document(metadata={'page': 3, 'total_pages': 11, 'source': '3.health_supplements_side_effects.pdf', 'moddate': '2024-10-21T14:37:42+00:00', 'creationdate': '', 'producer': 'iLovePDF', 'creator': 'PyPDF', 'page_label': '4'}, page_content='additional two studies have suggested increased mortality in smokers consuming β-carotene \nsupplements (18,19). Excess vitamin A supplementation has been suggested to be associated \nwith adverse effects on bone health including low bone mineral density and increased \nfracture risk (20). In addition, women consuming large amounts of vitamin A supplements \nduring pregnancy have been reported to have increased incidence of congenital \nabnormalities (21). There is also a case report of intra-hepatic cholestasis in a patient with \nchronic hypervitaminosis A after 12 years of supplement consumption which resolved after \nsupplements were ceased (22). In addition to toxicity from excess vitamin consumption,'),
 Document(metadata={'creationdate': '', 'p

In [32]:
question = "What programming languages do data scientists use?"
retrieved_docs = retriever.invoke(question)
retrieved_docs

[Document(metadata={'total_pages': 11, 'creator': 'PyPDF', 'source': '3.health_supplements_side_effects.pdf', 'page_label': '4', 'creationdate': '', 'page': 3, 'producer': 'iLovePDF', 'moddate': '2024-10-21T14:37:42+00:00'}, page_content='additional two studies have suggested increased mortality in smokers consuming β-carotene \nsupplements (18,19). Excess vitamin A supplementation has been suggested to be associated \nwith adverse effects on bone health including low bone mineral density and increased \nfracture risk (20). In addition, women consuming large amounts of vitamin A supplements \nduring pregnancy have been reported to have increased incidence of congenital \nabnormalities (21). There is also a case report of intra-hepatic cholestasis in a patient with \nchronic hypervitaminosis A after 12 years of supplement consumption which resolved after \nsupplements were ceased (22). In addition to toxicity from excess vitamin consumption,'),
 Document(metadata={'producer': 'iLovePDF'

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [39]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub

In [40]:
prompt = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
    Question: {question} 
    Context: {context} 
    Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [36]:
question = "Why are omega-3 fatty acids considered essential for humans??"

In [42]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [44]:
question = "Why are omega-3 fatty acids considered essential for humans?"
response = rag_chain.invoke(question)

print(response)

* Omega-3 fatty acids are considered essential for humans because they cannot be produced by the body and must be provided through the diet.
* They have been linked to several health benefits, including anti-cancer and cardioprotective effects, although the therapeutic benefits on cardiovascular diseases are still controversial.
* A study on Greenlanders found that a diet rich in polyunsaturated omega-3 fatty acids was associated with lower levels of plasma total lipids, plasma cholesterol, and plasma triglycerides, and


In [45]:
prompt2 = hub.pull("hwchase17/openai-tools-agent")

In [49]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "Why are omega-3 fatty acids considered essential for humans?"
response = rag_chain.invoke(question)
print(response)

* Omega-3 fatty acids are considered essential for humans because they cannot be produced by the body and must be provided through the diet.
* They have been linked to various health benefits, including anti-cancer and cardioprotective effects, although the therapeutic benefits on cardiovascular diseases are still controversial.
* A study on Greenlanders found that a diet rich in polyunsaturated omega-3 fatty acids was associated with lower levels of plasma total lipids, plasma cholesterol, and plasma triglycerides, and


In [51]:
question = "What programming languages do data scientists use?"
response = rag_chain.invoke(question)
print(response)

* I don't know what programming languages data scientists use based on the provided context. The context appears to be related to health supplements and their side effects, and does not mention programming languages or data science.


In [ ]:
@tool
def wikipedia_search(query):
    """
    Search wikipedia for general information.
    
    Args:
    query: The search query
    """
    
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(lang="en",
                                                                  top_k_results=3))
    response = wikipedia.invoke(query)
    return response